In [1]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import pickle as pkl

np.random.seed(42)

data_dir = "data/framing_politicians/morals/"
target = "all"  # or single instance like BLM
word2vec_file = "glove.840B.300d_w2v.txt"
word2vec_restricted_file = "glove.840B.300d_w2v_restricted.txt"
quick_run = False  # use it to limit word2vec loading, to quickly run results, final runs showed be set to False

# Here we run the frame axis against Twitter Data

The data is assumed to be already aggregated.  

The following resources are needed:
- Tweets for evaluation
- Annotations of Tweets
- Word2Vec Model
- FrameAxis with Moral Dictionary

The results shall reflect the results of the moral framing paper, but might be a bit different due to the rehydration process for twitter data.

In [2]:
tweets_df = pd.read_json(data_dir + target + ".jsonl", lines=True).set_index("id")
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21364 entries, 505332707002683392 to 265494444399263744
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 21364 non-null  datetime64[ns, UTC]
 1   id_str                     21364 non-null  int64              
 2   full_text                  21364 non-null  object             
 3   truncated                  21364 non-null  bool               
 4   display_text_range         21364 non-null  object             
 5   entities                   21364 non-null  object             
 6   source                     21364 non-null  object             
 7   in_reply_to_status_id      2343 non-null   float64            
 8   in_reply_to_status_id_str  2343 non-null   float64            
 9   in_reply_to_user_id        2983 non-null   float64            
 10  in_reply_to_user_id_str    2983 non-null

In [3]:
moral_df = pd.read_csv(data_dir + target + "_morals.csv").set_index("id")
moral_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34987 entries, 521033092132503552 to 265600333068238849
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   care         6468 non-null   float64
 1   purity       3149 non-null   float64
 2   subversion   6908 non-null   float64
 3   loyalty      6276 non-null   float64
 4   harm         9203 non-null   float64
 5   cheating     8168 non-null   float64
 6   fairness     6112 non-null   float64
 7   non-moral    24404 non-null  float64
 8   betrayal     5362 non-null   float64
 9   authority    6485 non-null   float64
 10  degradation  5002 non-null   float64
 11  nh           1 non-null      float64
 12  nm           109 non-null    float64
dtypes: float64(13)
memory usage: 3.7 MB


In [4]:
tweets_df = tweets_df.merge(moral_df, left_index=True, right_index=True)
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21602 entries, 20 to 844281435922075648
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 21602 non-null  datetime64[ns, UTC]
 1   id_str                     21602 non-null  int64              
 2   full_text                  21602 non-null  object             
 3   truncated                  21602 non-null  bool               
 4   display_text_range         21602 non-null  object             
 5   entities                   21602 non-null  object             
 6   source                     21602 non-null  object             
 7   in_reply_to_status_id      2353 non-null   float64            
 8   in_reply_to_status_id_str  2353 non-null   float64            
 9   in_reply_to_user_id        3003 non-null   float64            
 10  in_reply_to_user_id_str    3003 non-null   float64      

In [5]:
# We see that there can be overlap
# We will filter those out
assert len(tweets_df[(tweets_df["care"] >= 2) & (tweets_df["harm"] >= 2)][["care", "harm"]]) > 0
assert len(tweets_df[(tweets_df["purity"] >= 2) & (tweets_df["degradation"] >= 2)][["purity", "degradation"]]) > 0
assert len(tweets_df[(tweets_df["authority"] >= 2) & (tweets_df["subversion"] >= 2)][["authority", "subversion"]]) > 0
assert len(tweets_df[(tweets_df["loyalty"] >= 2) & (tweets_df["betrayal"] >= 2)][["loyalty", "betrayal"]]) > 0
assert len(tweets_df[(tweets_df["fairness"] >= 2) & (tweets_df["cheating"] >= 2)][["fairness", "cheating"]]) > 0

# Load Model and FrameAxis approach

In [6]:
if quick_run:
    model = KeyedVectors.load_word2vec_format(word2vec_file, limit=5000)
else:
    model = KeyedVectors.load_word2vec_format(word2vec_file)

## Load other data

In [18]:
%run frame_axis.py

In [19]:
fs = FrameSystem.load("moral.pkl")

In [20]:
fs.frame_axes

{'care/harm': <__main__.FrameAxis at 0x7fe40da8bd90>,
 'fairness/cheating': <__main__.FrameAxis at 0x7fe40da8bf70>,
 'loyalty/betrayal': <__main__.FrameAxis at 0x7fe40da8bfa0>,
 'authority/subversion': <__main__.FrameAxis at 0x7fe40da8beb0>,
 'sanctity/degradation': <__main__.FrameAxis at 0x7fe40da8b850>}

# Transform

In [21]:
# fs = FrameSystem(frame_axes)
tweets_df = fs.transform_df(tweets_df, "full_text", model)

# Reproduce

## Train on specific features only

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def axis_classification_report_individual(pos, neg, axis_bias, axis_inte):
    return axis_classification_report(pos, neg, axis_cols=[axis_bias, axis_inte])  # important only take the two dimension in question here

def axis_classification_report(pos, neg, axis_cols):
    """
    Peforms the classification and returns the report.
    The parameter axis_cols defines the columns to train on.
    """
    tweets_df_axis = tweets_df[(tweets_df[pos] > 2) | (tweets_df[neg] > 2)]
    X_axis = tweets_df_axis[axis_cols].values
    Y_axis =  tweets_df_axis[pos].values
    Y_axis = np.where(Y_axis >= 2, 1, 0)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_axis, Y_axis, test_size=0.25, shuffle=True)
    classifier = LogisticRegression(class_weight="balanced").fit(X_train, Y_train)  # penalty="l1", solver="saga"
    
    Y_pred = classifier.predict(X_test)
    print(f"{pos} - {neg}")
    print(classification_report(Y_test, Y_pred))
    return classification_report(Y_test, Y_pred, output_dict=True)

In [39]:
auth_res = axis_classification_report_individual("authority", "subversion", "auth_bias", "auth_inte")
fair_res = axis_classification_report_individual("fairness", "cheating", "fair_bias", "fair_inte")
care_res = axis_classification_report_individual("care", "harm", "care_bias", "care_inte")
loya_res = axis_classification_report_individual("loyalty", "betrayal", "loya_bias", "loya_inte")
pure_res = axis_classification_report_individual("purity", "degradation", "sanc_bias", "sanc_inte")
pass  # supress dict ouput

authority - subversion
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        80
           1       0.74      0.74      0.74        58

    accuracy                           0.78       138
   macro avg       0.78      0.78      0.78       138
weighted avg       0.78      0.78      0.78       138

fairness - cheating
              precision    recall  f1-score   support

           0       0.69      0.67      0.68       254
           1       0.64      0.67      0.65       228

    accuracy                           0.67       482
   macro avg       0.67      0.67      0.67       482
weighted avg       0.67      0.67      0.67       482

care - harm
              precision    recall  f1-score   support

           0       0.92      0.79      0.85       288
           1       0.76      0.90      0.82       208

    accuracy                           0.84       496
   macro avg       0.84      0.85      0.84       496
weighted avg       

### Order as table

In [40]:
def combine_classification_reports(dict_list, name_list):
    res_df = pd.DataFrame(dict_list)
    all_res_df = pd.concat([
        res_df, 
        pd.json_normalize(res_df['0']).add_prefix("0_"),
         pd.json_normalize(res_df['1']).add_prefix("1_"),
         pd.json_normalize(res_df['macro avg']).add_prefix("macro_avg_"),
         pd.json_normalize(res_df['weighted avg']).add_prefix("weighted_avg_"),
    ], axis=1).drop(["0", "1", "macro avg", "weighted avg"], axis=1)
    all_res_df.index = name_list
    return all_res_df

In [41]:
# Compute averages
dict_list = [auth_res, fair_res, care_res, loya_res, pure_res]
name_list = ["auth", "fair", "care", "loya", "pure"]
all_res_df = combine_classification_reports(dict_list, name_list)
all_res_df.loc["avg"] = all_res_df.mean()
all_res_df[["weighted_avg_precision", "weighted_avg_recall", "weighted_avg_f1-score", "accuracy"]]

,weighted_avg_precision,weighted_avg_recall,weighted_avg_f1-score,accuracy
auth,0.782609,0.782609,0.782609,0.782609
fair,0.666880,0.665975,0.666195,0.665975
care,0.850251,0.836694,0.837762,0.836694
loya,0.767270,0.657407,0.684767,0.657407
pure,0.906359,0.902174,0.901529,0.902174
avg,0.794674,0.768972,0.774572,0.768972


In [42]:
print(all_res_df[["weighted_avg_precision", "weighted_avg_recall", "weighted_avg_f1-score", "accuracy"]].to_latex(float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
{} &  weighted\_avg\_precision &  weighted\_avg\_recall &  weighted\_avg\_f1-score &  accuracy \\
\midrule
auth &                   0.783 &                0.783 &                  0.783 &     0.783 \\
fair &                   0.667 &                0.666 &                  0.666 &     0.666 \\
care &                   0.850 &                0.837 &                  0.838 &     0.837 \\
loya &                   0.767 &                0.657 &                  0.685 &     0.657 \\
pure &                   0.906 &                0.902 &                  0.902 &     0.902 \\
avg  &                   0.795 &                0.769 &                  0.775 &     0.769 \\
\bottomrule
\end{tabular}



## Train on all available features

In [43]:
moral_cols = ["auth_bias", "auth_inte", "fair_bias", "fair_inte", "care_bias", "care_inte", "loya_bias", "loya_inte", "sanc_bias", "sanc_inte"]
auth_res = axis_classification_report("authority", "subversion", moral_cols)
fair_res = axis_classification_report("fairness", "cheating", moral_cols)
care_res = axis_classification_report("care", "harm", moral_cols)
loya_res = axis_classification_report("loyalty", "betrayal", moral_cols)
pure_res = axis_classification_report("purity", "degradation", moral_cols)
pass  # supress dict ouput

authority - subversion
              precision    recall  f1-score   support

           0       0.76      0.74      0.75        70
           1       0.74      0.76      0.75        68

    accuracy                           0.75       138
   macro avg       0.75      0.75      0.75       138
weighted avg       0.75      0.75      0.75       138

fairness - cheating
              precision    recall  f1-score   support

           0       0.75      0.74      0.75       260
           1       0.70      0.71      0.71       222

    accuracy                           0.73       482
   macro avg       0.73      0.73      0.73       482
weighted avg       0.73      0.73      0.73       482

care - harm
              precision    recall  f1-score   support

           0       0.86      0.83      0.85       283
           1       0.78      0.82      0.80       213

    accuracy                           0.83       496
   macro avg       0.82      0.83      0.82       496
weighted avg       

In [44]:
# Compute averages
dict_list = [auth_res, fair_res, care_res, loya_res, pure_res]
name_list = ["auth", "fair", "care", "loya", "pure"]
all_res_df = combine_classification_reports(dict_list, name_list)
all_res_df.loc["avg"] = all_res_df.mean()
all_res_df[["weighted_avg_precision", "weighted_avg_recall", "weighted_avg_f1-score", "accuracy"]]

,weighted_avg_precision,weighted_avg_recall,weighted_avg_f1-score,accuracy
auth,0.753940,0.753623,0.753623,0.753623
fair,0.728519,0.728216,0.728339,0.728216
care,0.828146,0.826613,0.827042,0.826613
loya,0.895062,0.888889,0.891147,0.888889
pure,0.880671,0.880435,0.880449,0.880435
avg,0.817268,0.815555,0.816120,0.815555


In [45]:
print(all_res_df[["weighted_avg_precision", "weighted_avg_recall", "weighted_avg_f1-score", "accuracy"]].to_latex(float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
{} &  weighted\_avg\_precision &  weighted\_avg\_recall &  weighted\_avg\_f1-score &  accuracy \\
\midrule
auth &                   0.754 &                0.754 &                  0.754 &     0.754 \\
fair &                   0.729 &                0.728 &                  0.728 &     0.728 \\
care &                   0.828 &                0.827 &                  0.827 &     0.827 \\
loya &                   0.895 &                0.889 &                  0.891 &     0.889 \\
pure &                   0.881 &                0.880 &                  0.880 &     0.880 \\
avg  &                   0.817 &                0.816 &                  0.816 &     0.816 \\
\bottomrule
\end{tabular}

